In [7]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [8]:
from selenium.webdriver.remote.webelement import WebElement

In [9]:
from selenium.webdriver.common.keys import Keys

In [10]:
import re

In [89]:
driver1 = webdriver.Chrome(ChromeDriverManager().install())
wait1 = WebDriverWait(driver1, 30)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\YH\.wdm\drivers\chromedriver\win32\95.0.4638.69\chromedriver.exe] found in cache
<ipython-input-89-e66cfd37675b>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(ChromeDriverManager().install())


In [12]:
import gzip

In [13]:
def readGz(path):
    for l in gzip.open(path, 'rt',encoding='utf-8'):
        yield eval(l)

In [14]:
test_recipes = []
for d in readGz("testRecipes.json.gz"):
    test_recipes.append(d)

In [15]:
test_recipes_name = []
for x in test_recipes:
    test_recipes_name.append(x['name'])

In [16]:
test_recipes_name_id = dict()
for r in test_recipes:
    test_recipes_name_id[r['name']] = r['recipe_id']

In [18]:
# 记录名字搜出来是oops的recipe
ambiguous_recipe = []

In [34]:
minutes_6000 = {}

In [28]:
def search(r):
    submit = wait1.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'body > div.fd-site > div.fd-site-wrapper > section.gk-header-wrap.Food\.com > header > div > div > div > nav:nth-child(2) > ul > li.v-nav-toggle.has-icon.gk-nav__search > a > i'))
        )
    submit.click()

    # input the search recipe
    # 找text box需要通过id去找 注意！！！
    inputElement = driver1.find_element_by_id("search-input")
    inputElement.send_keys(r)

    # hit enter after inputting the text
    inputElement.send_keys(Keys.RETURN)

In [29]:
search(test_recipes_name[6000])

<ipython-input-28-04e9c0791751>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = driver1.find_element_by_id("search-input")


In [45]:
def get_minutes(temp_r_name):
    try :
        driver1.implicitly_wait(1)
        driver1.find_element_by_css_selector('#gk-menu-search > div.fd-menu-inner > div.search-module-results.on > header > div > h3')
        ambiguous_recipe.append(temp_r_name)
    except:
        # make sure the driver wait for the whole page to be loaded
        driver1.implicitly_wait(2)
        # 注意下面这一行代码，如果没有上面那一行代码去设置等待时间，可能会报错，因为提取的太快了，需要等待全部load
        e = driver1.find_element_by_css_selector('#gk-menu-search > div.fd-menu-inner > div.search-module-results.on > div > div > div > div > div.tile-content > div > div > div.meta-data > div.cook-time')
        # get the entire thing of the corresponding div class
        temp = e.get_attribute('outerHTML')
        # using regular expression to find the text in between
        time = re.findall('[0-9]+[\s\.]*[0-9]*\s+[hm]*', temp)[0].strip()
        temp_r_id = test_recipes_name_id[temp_r_name]
        minutes[temp_r_id] = time

In [55]:
import time

In [58]:
ambiguous_recipe = []

In [59]:
executed = 0

In [94]:
for r in test_recipes_name[7800:8000]:
    try:
        
        search(r)
        
        # 这个地方注意从原来的implict wait换成这个time sleep是因为
        # implicit wait是如果找不到对应的element一直等到 多久throw exception，但是
        # 网页一旦加载 那个信息就会出来，只是不包含我们想要的有效字段，所以我们怎么判断有没有我们要的
        # 关键字都没用，因为间隔时间太短太短了，不足以让我们要的字段上传。
        time.sleep(1.5)
        
        e = driver1.find_element_by_css_selector('#searchModuleTitle')
        
        # get the entire thing of the corresponding div class
        temp = e.get_attribute('outerHTML')
        
        if '1 Result' in temp:
            driver1.implicitly_wait(2)
            
            e = driver1.find_element_by_css_selector('#gk-menu-search > div.fd-menu-inner > div.search-module-results.on > div > div > div > div > div.tile-content > div > div > div.meta-data > div.cook-time')
            # get the entire thing of the corresponding div class
            temp = e.get_attribute('outerHTML')
            # using regular expression to find the text in between
            t = re.findall('[0-9]+[\s\.]*[0-9]*\s+[hm]*', temp)[0].strip()
            temp_r_id = test_recipes_name_id[r]
            minutes_6000[temp_r_id] = t
        else:
            continue
            
        executed += 1
        print(executed)
        
    except:
        # find oops recipe
        ambiguous_recipe.append(r)

        executed += 1
        print(executed)
            

<ipython-input-28-04e9c0791751>:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  inputElement = driver1.find_element_by_id("search-input")
<ipython-input-94-bfb46a210d44>:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  e = driver1.find_element_by_css_selector('#searchModuleTitle')
<ipython-input-94-bfb46a210d44>:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  e = driver1.find_element_by_css_selector('#gk-menu-search > div.fd-menu-inner > div.search-module-results.on > div > div > div > div > div.tile-content > div > div > div.meta-data > div.cook-time')


1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266


In [95]:
len(minutes_6000.keys())

947

In [96]:
%store minutes_6000

Stored 'minutes_6000' (dict)


In [63]:
ambiguous_recipe

[]

In [100]:
minutes_6000_revised = dict()
for x,y in minutes_6000.items():
    temp = y.strip().split(' ')
    if 'h' in temp:
        minutes_6000_revised[x] = float(temp[0]) * 60
    elif 'm' in temp:
        minutes_6000_revised[x] = float(temp[0])
    else:
        print(x)

In [102]:
%store minutes_6000_revised

Stored 'minutes_6000_revised' (dict)


In [103]:
minutes_6000_to_csv_list = []
for x, y in minutes_6000_revised.items():
    minutes_6000_to_csv_list.append([x,y])

In [105]:
import csv

In [110]:
with open('actual_minutes_6000.csv', 'w') as f:
    write = csv.writer(f)
    write.writerows(minutes_6000_to_csv_list)